In [1]:
!pip install --upgrade pip --index-url https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple

Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple


In [2]:
!pip install --upgrade pandas --index-url https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple

Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple


In [3]:
!pip install --upgrade numpy --index-url https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple

Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple


In [4]:
# Mention all file names as string in the below list which are available at "raw_data_path" directory
data_path = data_path_save = f'external/artichauhan/STD/Data'

raw_data_path = f'{data_path}/raw_data'

preprocessed_data_fnames = ['preprocessed_raw_data.csv']

# Mention the bucket name where pre-processed data will be stored. Usually there will no change in bucket name
bucket='adl-core-sagemaker-studio'

In [5]:
#importing librarires
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import boto3
import io
import time
import tempfile
import boto3
import joblib

%matplotlib inline
pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')
s3 = boto3.client('s3')

In [6]:
def load_data_from_s3(bucket,raw_data_path,raw_data_fnames):
    s3 = boto3.client('s3')
    dataset_nrows = []
    for i, fname in enumerate(raw_data_fnames):
        if i==0:
            print(f'Reading file: {fname}')
            key = f'{raw_data_path}/{fname}'
            obj = s3.get_object(Bucket=bucket, Key=key)
            data = pd.read_csv(io.BytesIO(obj['Body'].read()))
            dataset_nrows.append(data.shape[0])
            print(f'\tFile read successfully | Shape: {data.shape}')
        else:
            print(f'Reading file: {fname}')
            key = f'{raw_data_path}/{fname}'
            obj = s3.get_object(Bucket=bucket, Key=key)
            data2 = pd.read_csv(io.BytesIO(obj['Body'].read()))
            data = data.append(data2,ignore_index=True)
            dataset_nrows.append(data2.shape[0])
            print(f'\tFile read successfully | Shape: {data2.shape}')

    if sum(dataset_nrows) == data.shape[0]:
        print(f'Data from all files loaded successfully | Final Shape: {data.shape}')
        return data.copy()
    else:
        print('There is discrepency in numbers')
        print(f'\tTotal number of rows combined in all files: {sum(dataset_nrows)}')
        print(f'\tAfter combining all files total number of rows are: {data.shape[0]}')
        return None

In [7]:
Hype_LOA_Active = load_data_from_s3(bucket,raw_data_path,preprocessed_data_fnames)

Reading file: preprocessed_raw_data.csv
	File read successfully | Shape: (1345841, 33)
Data from all files loaded successfully | Final Shape: (1345841, 33)


In [8]:
Hype_LOA_Active[['mapped_employment_status_code','mapped_employment_status_description']].value_counts()

mapped_employment_status_code  mapped_employment_status_description
ACTIVE                         Active                                  766290
Active                         Active                                  232087
LOA_WITH_PAY                   LOA - With Pay                           81270
LOA_NO_PAY                     LOA - No Pay                             68672
STD                            Disability - Short Term                  59512
LTD                            Disability - Long Term                   42083
LOA - Unpaid Leave             LOA - no Pay                             36486
LOA_FM                         LOA - Family Medical                     17784
LOA - Paid Leave               LOA - with Pay                           13755
LOA - STD                      Disability - Short Term                   8869
LOA - Unpaid w/ Benefits       LOA - no Pay                              5730
LOA_WC                         LOA - Workers Compensation                4

In [9]:
Hype_LOA_Active['platform_indicator_code'].unique()

array(['R3', 'R4', 'HM', nan, 'HWE4', 'DBE', 'DCE'], dtype=object)

In [10]:
Hype_LOA_Active['mapped_employment_status_code'] = Hype_LOA_Active['mapped_employment_status_code'].replace(['LOA - Workers Comp'], ['LOA_WC'])

In [11]:
Hype_LOA_Active['mapped_employment_status_code'] = Hype_LOA_Active['mapped_employment_status_code'].replace(['LOA_NO_PAY'], ['LOA - Unpaid Leave'])

In [12]:
Hype_LOA_Active['mapped_employment_status_code'] = Hype_LOA_Active['mapped_employment_status_code'].replace(['LOA_WITH_PAY'], ['LOA - Paid Leave'])

In [13]:
Hype_LOA_Active['mapped_employment_status_code'] = Hype_LOA_Active['mapped_employment_status_code'].replace(['LTD'], ['LOA - LTD'])

In [14]:
Hype_LOA_Active['mapped_employment_status_code'] = Hype_LOA_Active['mapped_employment_status_code'].replace(['STD'], ['LOA - STD'])

In [15]:
Hype_LOA_Active['mapped_employment_status_code'] = Hype_LOA_Active['mapped_employment_status_code'].replace(['ACTIVE'], ['Active'])

In [16]:
Hype_LOA_Active[['mapped_employment_status_code']].value_counts()

mapped_employment_status_code
Active                           998377
LOA - Unpaid Leave               105158
LOA - Paid Leave                  95025
LOA - STD                         68381
LOA - LTD                         44605
LOA_FM                            17784
LOA - Unpaid w/ Benefits           5730
LOA_WC                             5330
LOA_MLTR                           3986
Temporary                          1465
dtype: int64

Creating the Label(target column):

In [17]:
# create Lable column from mapped_employment_status_code
Other_list = ['Active', 'Temporary','LOA - Unpaid Leave', 'LOA_FM', 'LOA - Paid Leave',
               'LOA - Unpaid w/ Benefits', 'LOA_MLTR', 'LOA_WC', 'LOA - LTD']
STD_list = ['LOA - STD']

In [18]:
# Creating an empty column
Hype_LOA_Active['mapped_employment_status_code_label']=''

In [19]:
# Label Column
Hype_LOA_Active.loc[Hype_LOA_Active['mapped_employment_status_code'].isin(Other_list), 'mapped_employment_status_code_label'] = 0
Hype_LOA_Active.loc[Hype_LOA_Active['mapped_employment_status_code'].isin(STD_list), 'mapped_employment_status_code_label'] = 1

In [20]:
Hype_LOA_Active['mapped_employment_status_code_label'].unique()

array([0, 1], dtype=object)

In [21]:
Hype_LOA_Active['mapped_employment_status_code_label'].value_counts()

0    1277460
1      68381
Name: mapped_employment_status_code_label, dtype: int64

In [22]:
Hype_LOA_Active['mapped_employment_status_code_label'] = Hype_LOA_Active['mapped_employment_status_code_label'].astype('int64')

In [23]:
Hype_LOA_Active.shape

(1345841, 34)

In [24]:
Hype_LOA_Active.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345841 entries, 0 to 1345840
Data columns (total 34 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   person_internal_id                           1345840 non-null  object 
 1   client_id                                    1345841 non-null  int64  
 2   udp_global_id                                1345841 non-null  object 
 3   age                                          1344842 non-null  float64
 4   gender                                       1340681 non-null  object 
 5   marital_status                               967801 non-null   object 
 6   is_union                                     1345841 non-null  object 
 7   primary_language_code                        340219 non-null   object 
 8   country_description                          1261252 non-null  object 
 9   state                                        1

In [25]:
col1 = Hype_LOA_Active.columns

Removing columns with more than 70% null values:

In [26]:
perc = 70.0
min_count =  int(((100-perc)/100)*Hype_LOA_Active.shape[0] + 1)
Hype_LOA_Active = Hype_LOA_Active.dropna( axis=1, 
                thresh=min_count)

In [27]:
Hype_LOA_Active.shape

(1345841, 30)

In [28]:
col2 = Hype_LOA_Active.columns

In [29]:
tf = col1.isin(col2)

In [30]:
df = pd.DataFrame({'col1':col1,'TF':tf})

In [31]:
# These columns have been removed where they had more then 30% null values
df[df['TF']==False]

,col1,TF
7,primary_language_code,False
19,subsidiary_code,False
20,subsidiary_description,False
32,annual_benefits_base_rate,False


Data cleaning: columns are having mutiple entries for a same text, hence reducing them to individual entries:

In [32]:
Divorce_list = ['Divorced', 'Divorced_United_States_of_America', 'Separated_United_States_of_America', \
                           'Divorced_USA', 'USA_Divorced', 'USA_Separated', 'Separated_USA', 'Separated USA', 'Divorced USA', \
                           'Legally_Separated_United_States_of_America', 'USA-Divorced', 'Widowed_United_States_of_America', 'USA-Separated','Widowed_USA','PR_Divorced']
Single_list = ['S', 'Single', 'Single_United_States_of_America', 'Single_USA', 'USA-Single', 'USA_Single', 'Single USA',\
                                           'S-USA', 'USA-Single, PR_Single', 'PR_Single']   
Unknown_list = ['Unknown_USA', 'Unknown_United_States_of_America', 'Not_Indicated_United_States_of_America', \
                  'PR_Living Together','USA_Living together','USA_Not Disclosed', 'USA_Not Disclosed','Living_Together_United_States_of_America',
                'Domestic_Partner _United_States_America', 'Civil P_United_States_of_America']
Married_list = ['M','Married_United_States_of_America','Married_USA','Married USA','Married','M_USA','M-USA','USA-Married','USA_Married','Married_USA','Maried','USA-Married/ Civil Partnership','Domestic_Partner_United_States_of_America','Partnered_United_States_of_America','Co-Habiting_United_States_of_America','Partnered USA','D-USA']
values = ['Single_Canada','Single_MEX', 'Single_DEU','Head_of_Household_USA','DE_FACTO','O-USA','MI_NOT_DISCLOSED','Domestic Partner', 'MARITAL_STATUS-3-321', 'Single_COL', 'PR_Partnered','Divorced_United_Kingdom','Single_MEX','MARITAL_STATUS-6-321','Unknown_United_States_of_America','Not_Indicated_United_States_of_America','Not_Disclosed_United_States_of_America','IN_Single',
          'Unknown_USA','MARITAL_STATUS-3-321','Married_MEX', 'S-HKG',
       'Unknown_New_Zealand', 'Unknown_Korea_Republic_of',
       'Married_Saudi_Arabia', 'Single_United Kingdom', 'Single_Thailand',
       'Unknown_United Kingdom', 'Unknown_India', 'Married_Taiwan',
       'Single_Taiwan', 'Single_Hong_Kong', 'Single_Korea_Republic_of', 'P-USA', 'Married_BRA', 'Married_DEU',
       'Unknown_SWE','Single_DOM',
       'Married_CHN', 'Married_ESP', 'Single_THA',
       'Married_Kenya', 'Unknown_Taiwan', 'Married_China',
       'Married_New_Zealand', 'Single0_Indonesia', 'Single_Spain',
       'Unknown_Thailand', 'Unknown_Spain', 'Married_Malaysia',
       'Married2_Indonesia', 'Married_Thailand','MARITAL_STATUS-3-40', 'Married_United_Kingdom',
       'Unknown_Puerto Rico','Single_United Arab Emirates', 'Divorced_Hong_Kong',
       'Civil_Partnership_MEX', 'Married_SGP', 'Unknown_AUS',
        'Single_SGP', 'Married_Spain',
       'Divorced_Singapore', 'Divorced_Canada', 'Married_Italy',
       'Married_MEX', 'Married_AUS', 'Single_ESP', 'Unknown_JPN',
       'PR_Married', 'M-GBR', 'Married_Switzerland', 'Domestic Partner',
       'W-USA', 'DE_FACTO', 'PR_Widowed', 'MI_NOT_DISCLOSED',
       'Married_CAN', 'Divorced_DEU', 'Dissolved_Civil_Partnership_MEX',
       'Single_FRA', 'Single_COL', 'Single_Puerto Rico', 'MARITAL_STATUS-6-301', 'MARITAL_STATUS-6-322', 'Hd Hsehld_United_States_of_America',
       'Common_Law_United_States_of_America', 'Married_Ireland',
       'Common_law_Canada', 'Married_Canada','Domestic_Partner_Canada',
       'U-USA','MARITAL_STATUS-3-323', 'USA-Cohabit', 'Married0_Indonesia', 'Married_Korea_Republic_of', 'Single_China',
       'Married1_Indonesia', 'Single_CAN', 'Married_SWE',
       'Single_Belgium', 'Married_United Kingdom', 'Married_Puerto Rico',
       'Married_ITA', 'Single_BEL', 'Married_BEL', 'Single_Lebanon',
       'Unknown_Kenya', 'M-DEU', 'Unknown_GBR', 'Civil Partner_Belgium',
       'Married_IRL', 'USA-Civil Partnership', 'Married_Hong_Kong',
       'Domestic Partner Civil Union_United Kingdom',
       'Civil_Partnership_COL', 'Unknown_Canada', 'Civil_Partnership_USA',
       'Single_JPN', 'Married_MYS', 'Single_IRL', 'Single_New_Zealand',
       'Married_Belgium', 'Civil_Partnership_United_States_of_America',
       'Married_COL', 'Common_Law_COL', 'Married_Austria', 'Single_GBR',
       'Married_GBR', 'Single_SWE', 'Married_JPN', 'Unknown_MLT',
       'Married_NZL', 'Unknown_CHN', 'PR_Not Disclosed', 'R-USA', 'RDP',
       'S-GBR', 'Married_Puerto_Rico', 'Married_Singapore', 'S-CHN',
       'Married_United Arab Emirates', 'M-SGP', 'Single_Singapore',
       'Separated_MLT', 'USA_Common-law', 'C-USA', 'S-ARE', 'S-SGP',
       'Single_CHN', 'Single_United_Kingdom', 'M-CAN', 'M-IND',
       'Single_BRA', 'Divorced_ESP', 'M-AUS', 'USA-Widowed',
       'PRI-Married', 'Single_Argentina', 'Unspecified_United_Kingdom',
       'Domestic_Partnered_United_States_of_America',
       'Co-Habiting_United_Kingdom', 'Married_Turkey',
       'Registered_Partnership_United_States_of_America',
       'Unknown_Malaysia', 'USA-Unknown', 'USA_Widowed', 'M-HKG']

In [33]:
Hype_LOA_Active.loc[Hype_LOA_Active['marital_status'].isin(Divorce_list), 'marital_status'] = 'Divorced'
Hype_LOA_Active.loc[Hype_LOA_Active['marital_status'].isin(Single_list), 'marital_status'] = 'Single'
Hype_LOA_Active.loc[Hype_LOA_Active['marital_status'].isin(Married_list), 'marital_status'] = 'Married'
Hype_LOA_Active.loc[Hype_LOA_Active['marital_status'].isin(Unknown_list), 'marital_status'] = 'unknown'

In [34]:
Hype_LOA_Active = Hype_LOA_Active[Hype_LOA_Active.marital_status.isin(values) == False]

In [35]:
Hype_LOA_Active['mapped_fullpart_code'] = Hype_LOA_Active['mapped_fullpart_code'].replace(['FT','PT'],['FLTM', 'PRTM'])
fullpart_code_remove_list = ['DNM', 'ERROR']
Hype_LOA_Active = Hype_LOA_Active[Hype_LOA_Active.mapped_fullpart_code.isin(fullpart_code_remove_list) == False]

In [36]:
Hype_LOA_Active['mapped_hourly_salary_code'] = Hype_LOA_Active['mapped_hourly_salary_code'].replace(['S', 'H'],['SLRY', 'HRLY'])
hr_code_remove_list = ['DNM', 'A', 'E', '1', 'L', 'N']
Hype_LOA_Active = Hype_LOA_Active[Hype_LOA_Active['mapped_hourly_salary_code'].isin(hr_code_remove_list) == False]

In [37]:
male = ['M', 'Male', 'male', 'Gender_Male']
female = ['F', 'Female', 'female', 'Gender_Female']
unknown = ['U', 'Unknown', 'Not_declared', 'D', 'Decline to answer', 'Undeclared',
           'Not specified', 'Declined to State', 'N', 'O', 'Not Declared','X','I do not wish to provide this information']

In [38]:
Hype_LOA_Active.loc[Hype_LOA_Active['gender'].isin(male), 'gender'] = 'male'
Hype_LOA_Active.loc[Hype_LOA_Active['gender'].isin(female), 'gender'] = 'female'
Hype_LOA_Active.loc[Hype_LOA_Active['gender'].isin(unknown), 'gender'] = 'unknown'

In [39]:
# #Saving the Rollup data for EDA
# Hype_LOA_Active.to_csv('Hype_LOA_Active.csv')

In [40]:
#Let's drop some columns which are not required
# Removing some id columns, state , date and description columns
# mapped_permanent_temporary_code and mapped_flex_status_code is having around 50% of null values hence dropping (EDA)
# country_description having only single value, won't  make much diff
# base_pay_regular_payrate_amount is highly correlated with base_pay_regular_expectedannualsalary, hence dropping base_pay_regular_payrate_amount
# base_pay_regular_expectedannualsalary_range - will create another range col and use that
# mapped_employment_status_description as we have label column created from this

drop_list = ['udp_global_id', 'state', 'mapped_fullpart_description', \
             'mapped_permanent_temporary_description', 'mapped_hourly_salary_description', 'mapped_flex_status_description', 
             'original_hire_date', 'rehire_date', 'termination_date','base_pay_regular_frequency_description',
             'base_pay_regular_expectedannualsalary_range','mapped_employment_status_description',
             'base_pay_regular_payrate_amount','country_description','mapped_flex_status_code',
             'mapped_permanent_temporary_code','base_pay_regular_frequency_code']



In [41]:
Hype_LOA_Active.drop(columns = drop_list, axis=1, inplace = True)

#### Categorical columns missing value imputation

In [42]:
#extract object columns and fill missing values
def fill_na_object(Data):
    object_col = Data.select_dtypes('object').columns
    for i in object_col:
        Data[i].fillna('unknown',inplace=True)
    return Data

In [43]:
#imputing missing values
Hype_LOA_Active = fill_na_object(Hype_LOA_Active)

In [44]:
# checking if missing values in categorical columns got imputed
Hype_LOA_Active.isna().sum()

person_internal_id                            0
client_id                                     0
age                                         998
gender                                        0
marital_status                                0
is_union                                      0
platform_indicator_code                       0
mapped_employment_status_code                 0
mapped_fullpart_code                          0
mapped_hourly_salary_code                     0
is_rehire                                     0
base_pay_regular_expectedannualsalary    278611
mapped_employment_status_code_label           0
dtype: int64

In [45]:
Hype_LOA_Active['mapped_employment_status_code_label'].value_counts()

0    1267435
1      67605
Name: mapped_employment_status_code_label, dtype: int64

In [46]:
Hype_LOA_Active.shape

(1335040, 13)

#### Splitting data set into train and test split

In [47]:
# # Separating the independent variables from dependent variables
# X = Hype_LOA_Active.drop(columns=['mapped_employment_status_code_label'], axis=1)
# y = Hype_LOA_Active['mapped_employment_status_code_label']

# #Split train-test data
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y,shuffle=True, random_state=42)


In [48]:
# print('X_train: ',X_train.shape)
# print('X_test: ',X_test.shape)
# print('y_train: ',y_train.shape)
# print('y_test: ',y_test.shape)

In [49]:
#Split train-test data
from sklearn.model_selection import train_test_split

train_set,test_set = train_test_split(Hype_LOA_Active, test_size = 0.20, 
                                   stratify=Hype_LOA_Active['mapped_employment_status_code_label'],shuffle=True, random_state=42)


In [50]:
print('train_set:',train_set.shape)
print('test_set:',test_set.shape)

train_set: (1068032, 13)
test_set: (267008, 13)


#### Uploading test set to s3

In [51]:
# test_set = pd.concat([X_test,y_test],axis=1)

In [52]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{data_path}')

test_set.to_csv(f's3://{bucket}/{data_path}/test_data/test_set.csv', index=False)

Uploading pre-processed data here -> s3://adl-core-sagemaker-studio/external/artichauhan/STD/Data


In [53]:
X_test = test_set.drop(['mapped_employment_status_code_label'],axis=1)
y_test = test_set['mapped_employment_status_code_label']

In [54]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{data_path}')

X_test.to_csv(f's3://{bucket}/{data_path}/test_data/rolledup_x_test.csv', index=False)

Uploading pre-processed data here -> s3://adl-core-sagemaker-studio/external/artichauhan/STD/Data


In [55]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{data_path}')

y_test.to_csv(f's3://{bucket}/{data_path}/test_data/y_test.csv', index=False)

Uploading pre-processed data here -> s3://adl-core-sagemaker-studio/external/artichauhan/STD/Data


#### Uploading train set to s3

In [56]:
# train_set = pd.concat([X_train,y_train],axis=1)

In [57]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{data_path}/')

train_set.to_csv(f's3://{bucket}/{data_path}/train_data/half_cleaned_train_set.csv', index=False)

Uploading pre-processed data here -> s3://adl-core-sagemaker-studio/external/artichauhan/STD/Data/


#### Imputing outliers

In [58]:
train_set.columns

Index(['person_internal_id', 'client_id', 'age', 'gender', 'marital_status',
       'is_union', 'platform_indicator_code', 'mapped_employment_status_code',
       'mapped_fullpart_code', 'mapped_hourly_salary_code', 'is_rehire',
       'base_pay_regular_expectedannualsalary',
       'mapped_employment_status_code_label'],
      dtype='object')

In [59]:
x = train_set.groupby(['mapped_employment_status_code'])['age'].quantile([0.01, 0.99]).unstack()
capping=x.reset_index()
capping

,mapped_employment_status_code,0.01,0.99
0,Active,18.00,75.0
1,LOA - LTD,29.00,89.0
2,LOA - Paid Leave,19.00,76.0
3,LOA - STD,22.00,75.0
4,LOA - Unpaid Leave,19.00,78.0
5,LOA - Unpaid w/ Benefits,17.00,73.0
6,LOA_FM,23.00,74.0
7,LOA_MLTR,20.00,64.0
8,LOA_WC,20.41,73.0
9,Temporary,18.00,75.0


In [60]:
train_set = train_set.merge(capping, how='left', on='mapped_employment_status_code')
train_set.shape

(1068032, 15)

In [61]:
train_set.rename(columns={0.01:'lower_age_cap',0.99:'upper_age_cap'},inplace=True)

In [62]:
train_set[['age','lower_age_cap','upper_age_cap']]

,age,lower_age_cap,upper_age_cap
0,52.0,18.0,75.0
1,39.0,18.0,75.0
2,28.0,18.0,75.0
3,39.0,18.0,75.0
4,55.0,18.0,75.0
...,...,...,...
1068027,64.0,18.0,75.0
1068028,62.0,19.0,76.0
1068029,25.0,20.0,64.0
1068030,29.0,18.0,75.0


In [63]:
train_set['age'] = np.where(train_set['age'] > train_set['upper_age_cap'], train_set['upper_age_cap'], train_set['age'])

In [64]:
train_set['age'] = np.where(train_set['age'] < train_set['lower_age_cap'], train_set['lower_age_cap'], train_set['age'])

In [65]:
# drop newly creating columns
col=['lower_age_cap','upper_age_cap']
train_set.drop(columns=col,axis=1,inplace=True)

In [66]:
train_cat_columns = train_set.select_dtypes(include=['object']).columns
train_num_columns = train_set.select_dtypes(include=['int']).columns
train_float_columns = train_set.select_dtypes(include=['float']).columns

In [67]:
print(train_cat_columns)
print(train_num_columns)
print(train_float_columns)

Index(['person_internal_id', 'gender', 'marital_status', 'is_union',
       'platform_indicator_code', 'mapped_employment_status_code',
       'mapped_fullpart_code', 'mapped_hourly_salary_code', 'is_rehire'],
      dtype='object')
Index(['client_id', 'mapped_employment_status_code_label'], dtype='object')
Index(['age', 'base_pay_regular_expectedannualsalary'], dtype='object')


#### Numeric Imputer for missing values

In [68]:
class CustomNumericImputer:
    
    def __init__(self):
        return None
    
    def fit(self, X, impute_cols, using_cols, method='median'):
        self.using_cols = using_cols
        self.impute_cols = impute_cols
        self.method = method
        cols = self.using_cols + self.impute_cols
        X_ = X[cols]
        
        if self.method == 'median':
            self.fit_values = X_.groupby(using_cols)[impute_cols].median().reset_index()
            impute_cols_rename_dict = dict([(x,x+'_median') for x in self.impute_cols])
            self.fit_values.rename(columns=impute_cols_rename_dict, inplace=True)
            self.fit_values.fillna(self.fit_values.median(), inplace=True)
        elif self.method == 'mean':
            self.fit_values = X_.groupby(using_cols)[impute_cols].mean().reset_index()
            impute_cols_rename_dict = dict([(x,x+'_mean') for x in self.impute_cols])
            self.fit_values.rename(columns=impute_cols_rename_dict, inplace=True)
            self.fit_values.fillna(self.fit_values.mean(), inplace=True)
        else: print('Method can be "median" or "mean"')
        self.new_cols = list(impute_cols_rename_dict.values())
        return self
    
    def fit_transform(self, X, impute_cols, using_cols, method='median'):
        self.fit(X, impute_cols, using_cols, method)
        
        cols = self.using_cols + self.impute_cols
        X_ = X[cols]
        X_ = X_.merge(self.fit_values, how='left', on=self.using_cols)
        for col, ncol in zip(self.impute_cols, self.new_cols):
            X_[col] = np.where(X_[col].isnull(), X_[ncol], X_[col])
            
        if self.method == 'median':
            return X_[self.impute_cols].fillna(X_[self.impute_cols].median())
        elif self.method == 'mean':
            return X_[self.impute_cols].fillna(X_[self.impute_cols].mean())
    
    def transform(self, X, impute_cols):
        cols = self.using_cols + self.impute_cols
        X_ = X[cols]
        X_ = X_.merge(self.fit_values, how='left', on=self.using_cols)
        for col, ncol in zip(self.impute_cols,self.new_cols):
            X_[col] = np.where(X_[col].isnull(), X_[ncol], X_[col])
            
        if self.method == 'median':
            return X_[self.impute_cols].fillna(X_[self.impute_cols].median())
        elif self.method == 'mean':
            return X_[self.impute_cols].fillna(X_[self.impute_cols].mean())
         

In [69]:
custom_numeric_imputer = CustomNumericImputer()

In [70]:
train_set[train_num_columns].describe()

,client_id,mapped_employment_status_code_label
count,1.068032e+06,1.068032e+06
mean,6.541099e+03,5.063893e-02
std,6.193052e+03,2.192594e-01
min,9.500000e+01,0.000000e+00
25%,1.762000e+03,0.000000e+00
50%,4.508000e+03,0.000000e+00
75%,8.673000e+03,0.000000e+00
max,9.999500e+04,1.000000e+00


In [71]:
train_set[train_float_columns].describe()

,age,base_pay_regular_expectedannualsalary
count,1.067228e+06,8.446420e+05
mean,4.292981e+01,6.670844e+04
std,1.455649e+01,4.609199e+05
min,1.700000e+01,0.000000e+00
25%,3.100000e+01,2.614560e+04
50%,4.200000e+01,4.567500e+04
75%,5.500000e+01,8.064160e+04
max,8.900000e+01,2.050000e+08


In [72]:
train_set[train_float_columns] = custom_numeric_imputer.fit_transform(X=train_set, impute_cols=train_float_columns.tolist(),
                                                              using_cols=['mapped_employment_status_code'], method='median')

In [73]:
custom_numeric_imputer.fit_values

,mapped_employment_status_code,age_median,base_pay_regular_expectedannualsalary_median
0,Active,41.0,48672.00
1,LOA - LTD,61.0,47777.60
2,LOA - Paid Leave,37.0,29120.00
3,LOA - STD,47.0,52936.00
4,LOA - Unpaid Leave,40.0,36910.95
5,LOA - Unpaid w/ Benefits,39.0,14586.00
6,LOA_FM,40.0,49836.80
7,LOA_MLTR,31.0,42660.80
8,LOA_WC,49.0,58635.20
9,Temporary,32.0,16526.58


In [74]:
# train_set.isna().sum()

In [75]:
# creating bins on the column base_pay_regular_expectedannualsalary and converting it into a range
bins = [-1, 19999, 39999, 59999, 79999, 99999, 174999,249999, 999999999999]
labels = ['<20,000', '20,000 - 39,999', '40,000 - 59,999', '60,000 - 79,999', '80,000 - 99,999', '100,000-174999', '175000-249999', '>250000']
train_set['base_pay_regular_expected_annual_salary_range'] = pd.cut(x=train_set['base_pay_regular_expectedannualsalary'], bins=bins, labels=labels)

In [76]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068032 entries, 0 to 1068031
Data columns (total 14 columns):
 #   Column                                         Non-Null Count    Dtype   
---  ------                                         --------------    -----   
 0   person_internal_id                             1068032 non-null  object  
 1   client_id                                      1068032 non-null  int64   
 2   age                                            1068032 non-null  float64 
 3   gender                                         1068032 non-null  object  
 4   marital_status                                 1068032 non-null  object  
 5   is_union                                       1068032 non-null  object  
 6   platform_indicator_code                        1068032 non-null  object  
 7   mapped_employment_status_code                  1068032 non-null  object  
 8   mapped_fullpart_code                           1068032 non-null  object  
 9   mapped_hourly

In [77]:
#Let's drop some columns which are not required
col = ['base_pay_regular_expectedannualsalary','mapped_employment_status_code']
train_set.drop(columns=col,axis=1,inplace=True)

In [78]:
train_set.shape

(1068032, 12)

In [79]:
train_set['age'] =train_set['age'].astype('int')

In [80]:
train_set['base_pay_regular_expected_annual_salary_range'] =train_set['base_pay_regular_expected_annual_salary_range'].astype('object')

### Exporting pre-processed train data to S3

In [81]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{data_path}')

train_set.to_csv(f's3://{bucket}/{data_path}/train_data/transformed_train_data.csv', index=False)

Uploading pre-processed data here -> s3://adl-core-sagemaker-studio/external/artichauhan/STD/Data


### Reading Test Data

In [82]:
test_data_path = f'{data_path}/test_data'
file_name = ['test_set.csv']
test_set_s3 = load_data_from_s3(bucket,test_data_path,file_name)

Reading file: test_set.csv
	File read successfully | Shape: (267008, 13)
Data from all files loaded successfully | Final Shape: (267008, 13)


#### Preprocessing of the Test Set

In [83]:
# seprating label column from the test set
test_set = test_set_s3.drop(['mapped_employment_status_code_label'],axis=1)
test_label = test_set_s3['mapped_employment_status_code_label']

In [84]:
test_set.columns

Index(['person_internal_id', 'client_id', 'age', 'gender', 'marital_status',
       'is_union', 'platform_indicator_code', 'mapped_employment_status_code',
       'mapped_fullpart_code', 'mapped_hourly_salary_code', 'is_rehire',
       'base_pay_regular_expectedannualsalary'],
      dtype='object')

In [85]:
test_set = test_set.merge(capping, how='left', on='mapped_employment_status_code')
test_set.shape

(267008, 14)

In [86]:
test_set.rename(columns={0.01:'lower_age_cap',0.99:'upper_age_cap'},inplace=True)

In [87]:
test_set['age'] = np.where(test_set['age'] > test_set['upper_age_cap'], test_set['upper_age_cap'], test_set['age'])

In [88]:
test_set['age'] = np.where(test_set['age'] < test_set['lower_age_cap'], test_set['lower_age_cap'], test_set['age'])

In [89]:
col=['lower_age_cap','upper_age_cap']
test_set.drop(columns=col,axis=1,inplace=True)

In [90]:
test_cat_columns = test_set.select_dtypes(include=['object']).columns
test_num_columns = test_set.select_dtypes(include=['int']).columns
test_float_columns = test_set.select_dtypes(include=['float']).columns

In [91]:
print(test_cat_columns)
print(test_num_columns)
print(test_float_columns)

Index(['person_internal_id', 'gender', 'marital_status', 'is_union',
       'platform_indicator_code', 'mapped_employment_status_code',
       'mapped_fullpart_code', 'mapped_hourly_salary_code', 'is_rehire'],
      dtype='object')
Index(['client_id'], dtype='object')
Index(['age', 'base_pay_regular_expectedannualsalary'], dtype='object')


#### Numeric Imputer

In [92]:
test_set[test_float_columns] = custom_numeric_imputer.transform(X=test_set, impute_cols=test_float_columns)
test_set.shape

(267008, 12)

In [93]:
test_set[test_float_columns].isna().sum()

age                                      0
base_pay_regular_expectedannualsalary    0
dtype: int64

In [94]:
# creating bins on the column base_pay_regular_expectedannualsalary and converting it into a range
bins = [-1, 19999, 39999, 59999, 79999, 99999, 174999,249999, 999999999999]
labels = ['<20,000', '20,000 - 39,999', '40,000 - 59,999', '60,000 - 79,999', '80,000 - 99,999', '100,000-174999', '175000-249999', '>250000']
test_set['base_pay_regular_expected_annual_salary_range'] = pd.cut(x=test_set['base_pay_regular_expectedannualsalary'], bins=bins, labels=labels)

In [95]:
test_set['age'] = test_set['age'].astype(int)

In [96]:
test_set['base_pay_regular_expected_annual_salary_range'] = test_set['base_pay_regular_expected_annual_salary_range'].astype('object')

In [97]:
test_set.shape

(267008, 13)

In [98]:
test_set.isna().sum()

person_internal_id                               0
client_id                                        0
age                                              0
gender                                           0
marital_status                                   0
is_union                                         0
platform_indicator_code                          0
mapped_employment_status_code                    0
mapped_fullpart_code                             0
mapped_hourly_salary_code                        0
is_rehire                                        0
base_pay_regular_expectedannualsalary            0
base_pay_regular_expected_annual_salary_range    0
dtype: int64

In [99]:
col = ['base_pay_regular_expectedannualsalary','mapped_employment_status_code']
test_set=test_set.drop(columns=col,axis=1)

In [100]:
test_set.shape

(267008, 11)

### Exporting transformed test data to S3

In [101]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{data_path}/test_data')
test_set.to_csv(f's3://{bucket}/{data_path}/test_data/transformed_test_data.csv', index=False)

Uploading pre-processed data here -> s3://adl-core-sagemaker-studio/external/artichauhan/STD/Data/test_data
